<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Data-Analysis-And-Machine-Learning-Projects/blob/master/5.%20Transportation%20and%20Logistics/Bike%20Sharing%20Prediction/Bike_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Enviorment

### Download Dataset

In [23]:
#!/bin/bash
!kaggle datasets download lakshmi25npathi/bike-sharing-dataset
!unzip bike-sharing-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/bike-sharing-dataset
License(s): unknown
bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  bike-sharing-dataset.zip
  inflating: Readme.txt              
  inflating: day.csv                 
  inflating: hour.csv                


### Install Libraries

In [24]:
%%bash
pip install -q xgboost lightgbm

### Import Libraries

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import (train_test_split,
                                     cross_val_score,
                                     GridSearchCV,
                                     RandomizedSearchCV)
from sklearn.preprocessing import (StandardScaler,
                                   LabelEncoder,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (mean_squared_error,
                             mean_absolute_error,
                             r2_score)
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor,
                              GradientBoostingRegressor)

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

### Import Dataset

In [56]:
df_day = pd.read_csv('day.csv')
df_hour = pd.read_csv('hour.csv')

# Analyzing Dataset

In [57]:
# ===================== #
# Analyze DataFrame
# ===================== #

def analyze_df(df, dataset_name, display_head=True, summary_stats=True):
    """
    Analyzes a given DataFrame and prints useful information about it.

    Parameters:
        df (pd.DataFrame): The DataFrame to analyze.
        dataset_name (str): Name of the dataset for display purposes.
        display_head (bool): If True, display the first few rows of the DataFrame.
        summary_stats (bool): If True, include summary statistics for numeric columns.

    Returns:
        None
    """
    # DataFrame Analysis
    print(f"{'-'*40}\nAnalyzing '{dataset_name}' DataFrame\n{'-'*40}\n")
    print(f"Number of Rows: {df.shape[0]} | Number of Columns: {df.shape[1]}")
    print("\nColumn Overview:")
    print(df.info())

    # Check Missing Values
    print(f"\n{'-'*40}\n Missing Values \n{'-'*40}\n")
    print("Missing Values: ")
    nans = df.isna().sum()
    if nans.sum() > 0:
        print(nans[nans > 0])
    else:
        print("No Missing Values...")

    # Check Dupes
    print(f"\n{'-'*40}\n Duplicated Rows \n{'-'*40}\n")
    dupes = df.duplicated().sum()
    print(f"Number of Duplicates: {dupes}")

    # Statistical Analysis
    print(f"\n{'-'*40}\n {dataset_name} Statistical Summary \n{'-'*40}\n")
    if summary_stats:
        print("Summary Statistics:")
        print(df.describe())

    if display_head:
        print(f"\n{'-'*40}\n DataFrame Sample \n{'-'*40}\n")
        print("\nFirst Few Rows:")
        return df.head()

    print(f"\n{'-'*40}\nAnalysis Complete for '{dataset_name}'\n{'-'*40}")

In [58]:
analyze_df(df_day, 'df_day')

----------------------------------------
Analyzing 'df_day' DataFrame
----------------------------------------

Number of Rows: 731 | Number of Columns: 16

Column Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [59]:
analyze_df(df_hour, 'df_hour', summary_stats=False)

----------------------------------------
Analyzing 'df_hour' DataFrame
----------------------------------------

Number of Rows: 17379 | Number of Columns: 17

Column Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


### Variables :

| **Column**      | **Description**                                                                 | **Data Type** | **Notes**                                    |
|------------------|---------------------------------------------------------------------------------|---------------|----------------------------------------------|
| `instant`       | Index of records. A unique identifier for each row.                            | `int64`       | Acts as a primary key or row ID.             |
| `dteday`        | Date of the record.                                                            | `object`      | Format: YYYY-MM-DD.                          |
| `season`        | Season of the year (1: Winter, 2: Spring, 3: Summer, 4: Fall).                 | `int64`       | Categorical variable.                        |
| `yr`            | Year of the record (0: 2011, 1: 2012).                                         | `int64`       | Categorical, representing two years.         |
| `mnth`          | Month of the year (1 to 12).                                                   | `int64`       | Numerical representation of months.          |
| `hr`            | Hour of the day (0 to 23).                                                     | `int64`       | Represents the time in 24-hour format.       |
| `holiday`       | Whether the day is a holiday (0: No, 1: Yes).                                  | `int64`       | Binary categorical variable.                 |
| `weekday`       | Day of the week (0: Sunday, 1: Monday, ..., 6: Saturday).                      | `int64`       | Categorical variable.                        |
| `workingday`    | Whether the day is a working day (0: No, 1: Yes).                              | `int64`       | Binary categorical variable.                 |
| `weathersit`    | Weather situation (1: Clear, 2: Mist, 3: Light Rain/Snow, 4: Heavy Rain/Snow). | `int64`       | Ordinal variable.                            |
| `temp`          | Normalized temperature in Celsius.                                             | `float64`     | Ranges between 0 and 1.                      |
| `atemp`         | Normalized "feeling" temperature in Celsius.                                   | `float64`     | Ranges between 0 and 1.                      |
| `hum`           | Normalized humidity.                                                           | `float64`     | Ranges between 0 and 1.                      |
| `windspeed`     | Normalized wind speed.                                                         | `float64`     | Ranges between 0 and 1.                      |
| `casual`        | Number of casual (non-registered) users.                                       | `int64`       | Count of casual bike rentals.                |
| `registered`    | Number of registered users.                                                    | `int64`       | Count of registered bike rentals.            |
| `cnt`           | Total count of bike rentals (casual + registered).                             | `int64`       | Target variable representing total rentals.  |

#### Notes:
- **Categorical Variables**: `season`, `yr`, `holiday`, `weekday`, `workingday`, `weathersit` are categorical but stored as integers.
- **Target Variable**: `cnt` is typically the variable to predict in bike rental prediction models.
- **Normalization**: `temp`, `atemp`, `hum`, and `windspeed` are normalized features, scaled to a range of 0 to 1.


### Statistical Summary Of `df_hour`

In [60]:
df_hour.describe().T

,count,mean,std,min,25%,50%,75%,max
instant,17379.0,8690.000000,5017.029500,1.00,4345.5000,8690.0000,13034.5000,17379.0000
season,17379.0,2.501640,1.106918,1.00,2.0000,3.0000,3.0000,4.0000
yr,17379.0,0.502561,0.500008,0.00,0.0000,1.0000,1.0000,1.0000
mnth,17379.0,6.537775,3.438776,1.00,4.0000,7.0000,10.0000,12.0000
hr,17379.0,11.546752,6.914405,0.00,6.0000,12.0000,18.0000,23.0000
holiday,17379.0,0.028770,0.167165,0.00,0.0000,0.0000,0.0000,1.0000
weekday,17379.0,3.003683,2.005771,0.00,1.0000,3.0000,5.0000,6.0000
workingday,17379.0,0.682721,0.465431,0.00,0.0000,1.0000,1.0000,1.0000
weathersit,17379.0,1.425283,0.639357,1.00,1.0000,1.0000,2.0000,4.0000
temp,17379.0,0.496987,0.192556,0.02,0.3400,0.5000,0.6600,1.0000


# Preprocessing

### Correct Column's Name

In [61]:
df_hour.rename(columns={
    'dteday': 'date',
    'yr': 'year',
    'mnth': 'month',
    'hr': 'hour',
    'holiday': 'is_holiday',
    'weekday': 'day_of_week',
    'workingday': 'is_workding_day',
    'weathersit': 'weather_situation',
    'atemp': 'feels_like_temp',
    'hum': 'humidity',
    'windspeed': 'wind_speed',
    'registered': 'registered_count',
    'cnt': 'total_count'
}, inplace=True)

In [62]:
df_hour.head(15)

,instant,date,season,year,month,hour,is_holiday,day_of_week,is_workding_day,weather_situation,temp,feels_like_temp,humidity,wind_speed,casual,registered_count,total_count
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,7,2011-01-01,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,8,2011-01-01,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,9,2011-01-01,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,10,2011-01-01,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


### Correcting Datatypes

In [63]:
#======================#
# Datetime             #
#======================#

df_hour['date'] = pd.to_datetime(df_hour['date'])

In [64]:
#======================#
# Categorical Features #
#======================#

categorical_features = ['season', 'year', 'month', 'hour', 'is_holiday', 'day_of_week', 'is_workding_day', 'weather_situation']

for feature in categorical_features:
    df_hour[feature] = df_hour[feature].astype('category')

In [67]:
#======================#
# Numerical Features   #
#======================#

numerical_features = ['temp', 'feels_like_temp', 'humidity', 'wind_speed',
                   'casual', 'registered_count', 'total_count']

for feature in numerical_features:
    df_hour[feature] = df_hour[feature].astype('float')

In [68]:
df_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   instant            17379 non-null  int64         
 1   date               17379 non-null  datetime64[ns]
 2   season             17379 non-null  category      
 3   year               17379 non-null  category      
 4   month              17379 non-null  category      
 5   hour               17379 non-null  category      
 6   is_holiday         17379 non-null  category      
 7   day_of_week        17379 non-null  category      
 8   is_workding_day    17379 non-null  category      
 9   weather_situation  17379 non-null  category      
 10  temp               17379 non-null  float64       
 11  feels_like_temp    17379 non-null  float64       
 12  humidity           17379 non-null  float64       
 13  wind_speed         17379 non-null  float64       
 14  casual